## News Classification | September 2017

In [14]:
#importing essential libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [18]:
#Calling the Webdriver and declaring the URL
driver=webdriver.Chrome(r"D:\downloadc\chrome downloads\New folder\chromedriver.exe")
url="https://timesofindia.indiatimes.com/archive/year-2017,month-9.cms"
driver.get(url)

In [19]:

div=driver.find_element_by_id("calenderdiv")
dates=[]
for i in div.find_elements_by_xpath(".//table/tbody/tr"):
    for k in i.find_elements_by_xpath(".//td/a"):
        date_exist=False
        date=k.get_attribute('href')
        ### searching if date already present in dates list
        for x in dates:
            if(x==date):date_exist=True
        ## adding date if date not present in list
        if date_exist == False:dates.append(date)

In [24]:
URL=[]
z=1
for j in dates:
    driver.get(j)
    table=driver.find_element_by_xpath("//table[@class='cnt'][2]")
    temp=table.find_elements_by_xpath(".//tbody/tr[2]/td[1]/div[3]/table/tbody/tr[2]/td")
    for i in temp[0].find_elements_by_xpath(".//span/a"):
        URL.append(i.get_attribute('href'))
    for i in temp[2].find_elements_by_xpath(".//span/a"):
        URL.append(i.get_attribute('href'))
    z=z+1

In [25]:
len(URL)

15000

In [29]:
def func(urls_):
    author=[]
    date=[]
    vertical=[]
    content=[]
    heading=[]
    for k in urls_:
        driver.get(k)
        try:
            txt=driver.find_element_by_xpath("//div[@class='as_byline']/div[2]").text
            if(txt.find('IST')==-1): author.append(txt)
            else: 
                author.append('')
        except: 
            try:author.append(driver.find_element_by_xpath("//div[@class='_3Mkg- byline']/span").text)
            except:author.append('')
        try:
            dt=driver.find_element_by_xpath("//div[@class='as_byline']/div[3]").text.replace('Updated:','').replace('Created:','')
            if(dt.find('IST')):date.append(dt)
            else: date.append('')
        except: 
            try:
                dt=driver.find_element_by_xpath("//div[@class='as_byline']/div[2]").text
                if(txt.find('IST')!=-1):date.append(dt.replace('Updated:','').replace('Created:',''))
                else:date.append('')  
            except:
                try:date.append(driver.find_element_by_xpath("//div[@class='_3Mkg- byline']").text[-23:])
                except:date.append('')     
        try: vertical.append(driver.find_element_by_xpath("//div[@class='navbdcrumb']/div[@class='wrapper']/ol/li[2]").text)
        except: vertical.append('')
    
        try: content.append(driver.find_element_by_xpath("//div[@class='article_content clearfix']/arttextxml/div[@class='section1']/div[@class='Normal']").text.replace('\n',''))
        except: 
            try:content.append(driver.find_element_by_xpath("//div[@class='ga-headlines']").text.replace('\n',''))
            except:content.append('')
                
        try: heading.append(driver.find_element_by_xpath("//div[@class='as_heading']/h1[@class='heading1']").text)
        except: 
            try:heading.append(driver.find_element_by_xpath("//div[@class='_2NFXP ']/h1[@class='_23498']").text)
            except:heading.append('')
    df=pd.DataFrame({'Date':date,'Author':author,'Vertical':vertical,'Headline':heading,'Description':content})
    return df


In [32]:
df=func(URL)
df.to_csv('sep2017.csv')